# ECBM E4040 - Assignment 2- Task 5: Kaggle Open-ended Competition

Kaggle is a platform for predictive modelling and analytics competitions in which companies and researchers post data and statisticians and data miners compete to produce the best models for predicting and describing the data.

If you don't have a Kaggle account, feel free to join at [www.kaggle.com](https://www.kaggle.com). To let the TAs do the grading more conveniently, please use Lionmail to join Kaggle and use UNI as your username.

Visit the website for this competition to join: 
[https://www.kaggle.com/t/8dd419892b1c49a3afb0cea385a7e677](https://www.kaggle.com/t/8dd419892b1c49a3afb0cea385a7e677)

Details about this in-class competition is shown on the website above. Please read carefully.

<span style="color:red">__TODO__:</span>
1. Train a custom model for the bottle dataset classification problem. You are free to use any methods taught in the class or found by yourself on the Internet (ALWAYS provide reference to the source). General training methods include:
    * Dropout
    * Batch normalization
    * Early stopping
    * l1-norm & l2-norm penalization
2. You'll be given the test set to generate your predictions (70% public + 30% private, but you don't know which ones are public/private). Achieve 70% accuracy on the public test set. The accuracy will be shown on the public leaderboard once you submit your prediction .csv file. 
3. (A) Report your results on the Kaggle, for comparison with other students' optimization results (you should do this several times). (C) Save your best model, using BitBucket, at the same time when you (B) submit the homework files into Courseworks. See instructions below. 

__Hint__: You can start from what you implemented in task 4. Another classic classification model named 'VGG16' can also be easily implemented.

## HW Submission Details:
There are three components to reporting the results of this task: 

**(A) Submission (possible several) of the .csv prediction file throught the Kaggle platform;**. You should start doing this VARY early, so that students can compare their work as they are making progress with model optimization.

**(B) Editing and submitting the content of this Jupyter notebook, through Courseworks; **
(i) The code for your CNN model and for the training function. The code should be stored in __./ecbm4040/neuralnets/kaggle.py__;
(ii) Print out your training process and accuracy __within this notebook__;

**(C) Submitting your best CNN model through instructor-owned private BitBucket repo.**

**Description of (C):** 
For this task, you will be utilizing bitbucket to save your model for submission. Bitbucket provides Git code managment. For those who are not familiar with git operations, please check [Learn Git with Bitbucket Cloud](https://www.atlassian.com/git/tutorials/learn-git-with-bitbucket-cloud) as reference.
**TAs will create a private Bitbucket repository for each student, with the write access. This repo will be owned by the instructors. Make sure to properly submit your model to that exact repository (submissions to your own private repository will not count)** Students need to populate the following file to provide instructors with bitbucket account information: https://docs.google.com/spreadsheets/d/1_7cZjyr34I2y-AD_0N5UaJ3ZnqdhYcvrdoTsYvOSd-g/edit#gid=0.

<span style="color:red">__Submission content:__ :</span>
(i) Upload your best model with all the data output (for example, __MODEL.data-00000-of-00001, MODEL.meta, MODEL.index__) into the  BitBucket. Store your model in the folder named "__KaggleModel__" within the BitBucket repository. 
Remember to delete any intermediate results, **we only want your best model. Do not upload any data files**. The instructors will rerun the uploaded best model and verify against the score which you reported on the Kaggle.



## Train your model here

In [2]:
import numpy as np
from scipy import misc
import imageio
from ecbm4040.neuralnets.kaggle import training
import tensorflow as tf
import pandas as pd

In [3]:
X_train = []
Y_train = []
for label in range(5):
    for pic in range(3000):
        pic_idx = str(label*3000 + pic)
        arr = imageio.imread('data/train_128/%s/%s.png'%(label,pic_idx)) 
        # X_train.append(arr)
        X_train.append(arr[::4,::4])
        Y_train.append(label)   


In [4]:
X_org = np.array(X_train)
Y_org = np.array(Y_train)

num_org = X_org.shape[0]
train_proportion = 0.75

In [5]:
sample_idxs = np.random.choice(num_org, int(num_org*train_proportion))
train_X = X_org[sample_idxs,:]
train_Y = Y_org[sample_idxs]
print(train_X.shape)
print(train_Y.shape)

(11250, 32, 32, 3)
(11250,)


In [6]:
val_X = X_org[-sample_idxs,:]
val_Y = Y_org[-sample_idxs]

In [7]:
tf.reset_default_graph()
training(train_X, train_Y, val_X, val_Y, 
         conv_featmap=[60,70],
         fc_units=[120,100],
         conv_kernel_size=[6,6],
         pooling_size=[2,2],
         l2_norm=0.01,
         seed=235,
         learning_rate=1e-2,
         epoch=20,
         batch_size=105,
         verbose=False,
         pre_trained_model=None)

Building my LeNet. Parameters: 
conv_featmap=[60, 70]
fc_units=[120, 100]
conv_kernel_size=[6, 6]
pooling_size=[2, 2]
l2_norm=0.01
seed=235
learning_rate=0.01
***************************************************8
Tensor("inputs/Placeholder:0", shape=(?, 32, 32, 3), dtype=float32)
(?, 32, 32, 3)
True
True
3 3 True
Pass0 (?, 32, 32, 60)
Pass1 (?, 16, 16, 60)
Pass2 (?, 16, 16, 60)
***************************************************8
Tensor("local_response_norm/LRN:0", shape=(?, 16, 16, 60), dtype=float32)
(?, 16, 16, 60)
True
True
60 60 True
Pass3 (?, 16, 16, 70)
Pass4 (?, 8, 8, 70)
Pass5 (?, 8, 8, 70)
Pass6 (?, 120)
number of batches for training: 107
epoch 1 
Best validation accuracy! iteration:100 accuracy: 48.82666666666667%
epoch 2 
Best validation accuracy! iteration:200 accuracy: 63.973333333333336%
epoch 3 
Best validation accuracy! iteration:300 accuracy: 68.32%
epoch 4 
Best validation accuracy! iteration:400 accuracy: 69.54666666666667%
epoch 5 
Best validation accuracy! iterati

## Save your best model

In [8]:
X_test = []
for pic in range(3500):
    pic_idx = str(pic)
    arr = imageio.imread('data/test_128/%s.png'%(pic_idx)) 
    X_test.append(arr[::4,::4])

In [9]:
from ecbm4040.neuralnets.kaggle import predict
tf.reset_default_graph()
kaggle_result = predict(X_test,"lenet_1509837508.meta")

INFO:tensorflow:Restoring parameters from model/lenet_1509837508
tf_input: inputs/Placeholder:0
[<tf.Operation 'inputs/Placeholder' type=Placeholder>, <tf.Operation 'inputs/Placeholder_1' type=Placeholder>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/shape' type=Const>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/min' type=Const>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/max' type=Const>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/RandomUniform' type=RandomUniform>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/sub' type=Sub>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/mul' type=Mul>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform' type=Add>, <tf.Operation 'conv_layer_0/conv_kernel_0' type=VariableV2>, <tf.Operation 'conv_layer_0/conv_kernel_0/Assign' type=Assign>, <tf.Operation 'conv_layer_0/conv_kernel_0/read' t

## Generate .csv file for Kaggle

In [10]:
# The following code snippet can be used to generate your prediction .csv file.
import csv
with open('predicted.csv','w',newline='') as csvfile:
    fieldnames = ['Id','label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()    
    for index,l in enumerate(kaggle_result):
        filename = str(index)+'.png'
        label = str(l)
        writer.writerow({'Id': filename, 'label': label})